# Test: hypothesis transition and memory state transition


In [1]:
# Section: Setup and Mock Classes

import os
import sys
import pathlib
import numpy as np
import numpy.testing as npt

# Robustly find the 'src' directory
current_dir = pathlib.Path.cwd()
src_path = None

# Check current dir, parent, and parent's parent for 'src'
for path in [current_dir, current_dir.parent, current_dir.parent.parent]:
    candidate = path / "src"
    if candidate.exists() and candidate.is_dir():
        src_path = str(candidate.resolve())
        break

if src_path:
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
    print(f"Added {src_path} to sys.path")
else:
    print("Warning: 'src' directory not found. Imports may fail.")

from Bayesian_state.problems.modules.hypo_transitions import FixedNumHypothesisModule
from Bayesian_state.problems.modules.memory import DualMemoryModule


class MockPartition:
    def __init__(self, similarity_matrix=None):
        self.similarity_matrix = similarity_matrix


class MockEngine:
    def __init__(self, set_size: int):
        self.set_size = set_size
        self.posterior = None
        self.prior = None
        self.hypotheses_mask = np.ones(set_size, dtype=float)
        self.partition = None
        self.state = {"fade": None, "static": None}


print('Setup complete.')

INFO:cat-learning:logger is running normally.


Added D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\src to sys.path
D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\logs\Run_20251203_173001.log
{'base_model': {'modules': {'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {}}}, 'agenda': ['likelihood_mod', '__self__']}, 'default_model': {'modules': {}, 'agenda': ['__self__']}, 'm_model': {'modules': {'likelihood_mod': {'class': 'src.Bayesian_state.problems.modules.likelihood.LikelihoodModule', 'kwargs': {'beta': 5.0}}, 'memory_mod': {'class': 'src.Bayesian_state.problems.modules.memory.DualMemoryModule', 'kwargs': {'w0': 0.8, 'gamma': 0.8}}}, 'agenda': ['likelihood_mod', 'memory_mod']}, 'pmh_model': {'modules': {'perception_mod': {'class': 'src.Bayesian_state.problems.modules.perception.PerceptionModule'}, 'hypo_transitions_mod': {'class': 'src.Bayesian_state.problems.modules.hypo_transitions.FixedNumHypothesisModule', 'kwargs': {'

### Test 1: Hypothesis Transition

In [2]:
# Section: Test 1 - Hypothesis Transition (Posterior -> Prior)
print("=== Test 1: Hypothesis Transition (Posterior -> Prior) ===")

# Create engine and set a known posterior
engine = MockEngine(set_size=6)
engine.posterior = np.array([0.6, 0.3, 0.1, 0.0, 0.0, 0.0], dtype=float)
# Make sure it sums to 1
engine.posterior = engine.posterior / engine.posterior.sum()
print(f"Initial Posterior (on all 6 hypos): {np.round(engine.posterior, 3)}")

# Define similarity matrix such that new hypo index 3 has known similarities to old [0,1,2]
sim = np.ones((6,6), dtype=float) / 3
# row for added hypothesis 3 -> sim to [0,1,2]
sim[3, 0] = 0.2
sim[3, 1] = 1.4
sim[3, 2] = 0.4
# attach partition
engine.partition = MockPartition(similarity_matrix=sim)

# Instantiate hypothesis module (use stable init to avoid random sampling side-effects)
hypo = FixedNumHypothesisModule(engine, fixed_hypo_num=3, init_strategy='stable')
# Force an artificial old->new active change: old [0,1,2] -> new [1,2,3]
hypo.old_active = np.array([0,1,2], dtype=int)
hypo.active = np.array([1,2,3], dtype=int)
print(f"Transition: Old Active {hypo.old_active} -> New Active {hypo.active}")
print(f"Added Hypothesis: 3, Removed Hypothesis: 0")
print(f"Similarity of Hypo 3 to Old Active [0, 1, 2]: {sim[3, [0,1,2]]}")

# Run the transition method
print("\nRunning _posterior_to_prior_transition()...")
hypo._posterior_to_prior_transition()

print(f"Engine.prior after transition: {np.round(engine.prior, 4)}")


print("\n--- old actives remain relatively stable ---")
print(f"{np.round(engine.posterior[hypo.old_active] / engine.prior[hypo.old_active], 4)}")

# --- Verification Logic ---
print("\n--- Verification ---")

# Compute expected new prior manually
old_indices = hypo.old_active
added_indices = np.array([3], dtype=int)
new_indices = hypo.active
current_posterior = engine.posterior.copy()

# initial new_prior copies current posterior
expected_prior = current_posterior.copy()

# compute weights using sim submatrix
sim_sub = sim[np.ix_(added_indices, old_indices)]  # shape (1,3)
row_sums = sim_sub.sum(axis=1, keepdims=True)
row_sums[row_sums == 0] = 1.0
weights = sim_sub / row_sums
print(f"Calculated Weights (Sim / RowSums): {weights}")

old_probs_active = current_posterior[old_indices]
if old_probs_active.sum() > 0:
    old_probs_active = old_probs_active / old_probs_active.sum()
print(f"Normalized Old Posterior on Active Set: {np.round(old_probs_active, 3)}")

added_probs = (weights @ old_probs_active).ravel()
print(f"Calculated Probability for Added Hypo 3 (before final norm): {added_probs}")
expected_prior[added_indices] = added_probs

# zero out removed (index 0) and keep survivors' original posterior
mask_new = np.zeros(6, dtype=float)
mask_new[new_indices] = 1.0
expected_prior *= mask_new
# normalize on new active set
if expected_prior.sum() > 0:
    expected_prior = expected_prior / expected_prior.sum()

print(f"Expected prior (manually computed): {np.round(expected_prior, 4)}")

# Assertions
npt.assert_allclose(engine.prior, expected_prior, rtol=1e-7, atol=1e-12)
print('\nTest 1 passed: engine.prior matches expected_prior')

=== Test 1: Hypothesis Transition (Posterior -> Prior) ===
Initial Posterior (on all 6 hypos): [0.6 0.3 0.1 0.  0.  0. ]
Transition: Old Active [0 1 2] -> New Active [1 2 3]
Added Hypothesis: 3, Removed Hypothesis: 0
Similarity of Hypo 3 to Old Active [0, 1, 2]: [0.2 1.4 0.4]

Running _posterior_to_prior_transition()...
Engine.prior after transition: [0.     0.4348 0.1449 0.4203 0.     0.    ]

--- old actives remain relatively stable ---
[ inf 0.69 0.69]

--- Verification ---
Calculated Weights (Sim / RowSums): [[0.1 0.7 0.2]]
Normalized Old Posterior on Active Set: [0.6 0.3 0.1]
Calculated Probability for Added Hypo 3 (before final norm): [0.29]
Expected prior (manually computed): [0.     0.4348 0.1449 0.4203 0.     0.    ]

Test 1 passed: engine.prior matches expected_prior


C:\Users\Ran\AppData\Local\Temp\ipykernel_26408\1791984420.py:37: RuntimeWarning: divide by zero encountered in divide
  print(f"{np.round(engine.posterior[hypo.old_active] / engine.prior[hypo.old_active], 4)}")


### Test 2: Memory State Transition Logic

This test verifies `_state_transition`.

**Goal:** Ensure the internal memory state (`static` and `fade`) is adjusted so that the resulting probability distribution matches the new `engine.prior`.

**Mechanism:**
- **Survivors (1, 2):** Used to calculate a `shift` factor. This shift aligns the scale of the memory state values (which are log-probs + arbitrary constant) to the new target log-probs.
- **Added (3):** Initialized to `target_log_prob + shift`.
- **Removed (0):** Set to `-inf`.

In [3]:
# Section: Test 2 - Memory State Transition (_state_transition)
print("\n=== Test 2: Memory State Transition (_state_transition) ===")

# Prepare a fresh engine to instantiate memory with an 'old' prior
engine2 = MockEngine(set_size=6)
old_active = np.array([0,1,2], dtype=int)
old_mask = np.zeros(6, dtype=float); old_mask[old_active] = 1.0
# Set engine2.hypotheses_mask so memory picks it up as the existing mask
engine2.hypotheses_mask = old_mask.copy()

# old prior: use posterior restricted to old_active (normalized)
prior_old = np.zeros(6, dtype=float)
prior_old[old_active] = np.array([0.6, 0.2, 0.1])
prior_old = prior_old / prior_old.sum()
engine2.prior = prior_old.copy()
print(f"Old Prior (on {old_active}): {np.round(prior_old[old_active], 3)}")

# Instantiate DualMemoryModule so its state is initialised from prior_old
mem = DualMemoryModule(engine2, w0=0.5, gamma=0.9)
# At init, mem.state['static'] and ['fade'] should be translate_to_log(prior_old, mask=old_mask)

# Now set engine2.prior to the new prior computed previously (from engine.prior)
# We'll reuse `engine.prior` from Test 1 as the target prior
engine2.prior = engine.prior.copy()
new_mask = mask_new.copy()  # from Test 1 active mask [1,2,3]
print(f"Target New Prior (on {np.where(new_mask)[0]}): {np.round(engine2.prior[new_mask.astype(bool)], 3)}")

# Keep a copy of pre-transition combined state for survivors
w0 = mem.w0
s_static = mem.state.get('static').copy()
s_fade = mem.state.get('fade').copy()
current_combined = w0 * s_static + (1 - w0) * s_fade

# Call the state transition
print("Running _state_transition()...")
mem._state_transition(new_mask)

# --- Verification Logic ---
print("\n--- Verification ---")

# 1) removed hypotheses (index 0) set to -inf in both state arrays
removed_idx = 0
assert np.isneginf(mem.state['static'][removed_idx]) and np.isneginf(mem.state['fade'][removed_idx]), 'Removed hypothesis not set to -inf'
print(f'Removed hypothesis {removed_idx} set to -inf as expected.')

# 2) added hypotheses get state = target_log + shift
# Compute expected target_log and shift
prior_new = engine2.prior.copy()
# compute target log (using memory.translate_to_log)
target_log = mem.translate_to_log(prior_new, mask=new_mask)

survivor_mask_bool = (new_mask.astype(bool)) & (old_mask.astype(bool))
# shift = mean(current_combined[survivors] - target_log[survivors])
shift_expected = np.mean(current_combined[survivor_mask_bool] - target_log[survivor_mask_bool])
print(f"Calculated Shift (from survivors {np.where(survivor_mask_bool)[0]}): {shift_expected:.4f}")

added_mask_bool = (new_mask.astype(bool)) & (~old_mask.astype(bool))
added_indices = np.where(added_mask_bool)[0]

for idx in added_indices:
    expected_val = target_log[idx] + shift_expected
    print(f"Hypo {idx} (Added): Target LogProb = {target_log[idx]:.4f}, Expected State = {expected_val:.4f}")
    print(f"               Actual Static = {mem.state['static'][idx]:.4f}, Actual Fade = {mem.state['fade'][idx]:.4f}")
    
    # check both static and fade
    npt.assert_allclose(mem.state['static'][idx], expected_val, rtol=1e-7, atol=1e-12)
    npt.assert_allclose(mem.state['fade'][idx], expected_val, rtol=1e-7, atol=1e-12)

print('\nTest 2 passed: memory state transition behavior matches expectation.')


=== Test 2: Memory State Transition (_state_transition) ===
Old Prior (on [0 1 2]): [0.667 0.222 0.111]
Target New Prior (on [1 2 3]): [0.435 0.145 0.42 ]
Running _state_transition()...

--- Verification ---
Removed hypothesis 0 set to -inf as expected.
Calculated Shift (from survivors [1 2]): -0.4684
Hypo 3 (Added): Target LogProb = -0.8668, Expected State = -1.3352
               Actual Static = -1.3352, Actual Fade = -1.3352

Test 2 passed: memory state transition behavior matches expectation.


D:\Research\nips_reviewing\CategoryLearning\StateBased\CategoryLearning\src\Bayesian_state\problems\modules\memory.py:214: RuntimeWarning: divide by zero encountered in log
  return np.log(clipped)


### Integrated Test

Simulates a full step where:
1. `FixedNumHypothesisModule` updates `engine.prior` based on hypothesis change.
2. `DualMemoryModule` updates its internal state based on the new `engine.prior`.
3. We verify that the memory state correctly reconstructs the `engine.prior`.

In [4]:
# Section: Integrated Test - Full Step Simulation
print("\n=== Integrated Test: Full Step Simulation ===")

# Create a shared engine for integrated test
en3 = MockEngine(set_size=6)
en3.posterior = np.array([0.6, 0.2, 0.1, 0.05, 0.03, 0.02])
en3.posterior /= en3.posterior.sum()

# Partition and hypothesis module
sim2 = np.zeros((6,6), dtype=float)
sim2[3, 0] = 0.1; sim2[3,1] = 0.7; sim2[3,2] = 0.2
en3.partition = MockPartition(similarity_matrix=sim2)

hypo2 = FixedNumHypothesisModule(en3, fixed_hypo_num=3, init_strategy='stable')
hypo2.old_active = np.array([0,1,2], dtype=int)
hypo2.active = np.array([1,2,3], dtype=int)

print("Step 1: Running Hypothesis Transition (Posterior -> Prior)...")
# run posterior->prior transition
hypo2._posterior_to_prior_transition()
print(f"Intermediate Engine Prior: {np.round(en3.prior, 4)}")

# Setup memory with prior corresponding to old_mask then transition it
engine_mem = MockEngine(set_size=6)
old_mask2 = np.zeros(6); old_mask2[[0,1,2]] = 1.0
engine_mem.hypotheses_mask = old_mask2.copy()
prior_old2 = np.zeros(6); prior_old2[[0,1,2]] = np.array([0.6,0.2,0.1]); prior_old2 /= prior_old2.sum()
engine_mem.prior = prior_old2.copy()
mem2 = DualMemoryModule(engine_mem, w0=0.5, gamma=0.9)

# Now set engine_mem.prior to the new prior from hypo2
engine_mem.prior = en3.prior.copy()
new_mask2 = np.zeros(6); new_mask2[[1,2,3]] = 1.0

print("Step 2: Running Memory State Transition...")
mem2._state_transition(new_mask2)

# Reconstruct posterior-like probabilities from memory state
log_posterior = mem2.w0 * mem2.state['static'] + (1 - mem2.w0) * mem2.state['fade']
recon = mem2.translate_from_log(log_posterior, mask=new_mask2)

print("\n--- Final Comparison ---")
print(f"Target Engine Prior:       {np.round(engine_mem.prior, 5)}")
print(f"Reconstructed from Memory: {np.round(recon, 5)}")

npt.assert_allclose(recon, engine_mem.prior, rtol=1e-6, atol=1e-10)
print('\nIntegrated test passed: reconstructed distribution matches engine.prior')


=== Integrated Test: Full Step Simulation ===
Step 1: Running Hypothesis Transition (Posterior -> Prior)...
Intermediate Engine Prior: [0.     0.3673 0.1837 0.449  0.     0.    ]
Step 2: Running Memory State Transition...

--- Final Comparison ---
Target Engine Prior:       [0.      0.36735 0.18367 0.44898 0.      0.     ]
Reconstructed from Memory: [0.      0.36735 0.18367 0.44898 0.      0.     ]

Integrated test passed: reconstructed distribution matches engine.prior
